### Energieversorgung Vertiefung Hausübung 1: Zuverlässigkeitskennzahlen

In [44]:
import pandas as pd
from tabulate import tabulate

In [45]:
# Eingabe Matrikelnummer
MatrNr = 11907067


# Berechnung IPS
def calculation_ips(Matrikelnummer):
    q = sum(int(digit) for digit in str(Matrikelnummer))
    return ((q * 5) % 40) * 10**-4


IPS = calculation_ips(MatrNr)


# Berechnung der Zuverlässigkeitskennzahlen
def calculate_kenngroessen(defizithaeufigkeit, ausfalldauer, last, laenge=1):
    H = defizithaeufigkeit * laenge
    NV = H * ausfalldauer
    P = H * last
    E = NV * last
    return H, NV, P, E

In [46]:
# Parameter Leitungslänge [km]
ltg_l = 5

# Parameter Ausfallhäufigkeit
h_ltg110 = 0.005 * ltg_l  # [1/a]
h_ltg20 = 0.001 * ltg_l  # [1/a]
h_T = 0.015  # [1/a]
h_TR = 0.00012 + IPS  # [1/a]
h_ls = 0.000259  # [1/a]
h_SS = 0.000148  # [1/a]

# Parameter Ausfalldauer
ta_ltg110 = 80  # [h]
ta_ltg20 = 26  # [h]
ta_T = 28  # [h]
ta_TR = 72  # [h]
ta_ls = 72  # [h]
ta_SS = 72  # [h]
ksT_UW = 0.25  # Korrekturschaltdauer im UW
ksT_aUW = 0.25  # Korrekturschaltdauer außerhalb vom UW

# Parameter Lasten [MW]
P_A = 5
P_GK = 5
P_B = 5
P_C = 5

In [47]:
# Berechnung für den ersten Fall: Aktuelle Situation, also der Großkunde wird nur vom UW1 mit der H-Schaltung versorgt
elemente_H_Schaltung = [
    {"name": "LTG1", "defizithaeufigkeit": h_ltg110 * ltg_l, "ausfalldauer": ta_ltg110},
    {"name": "LTG2", "defizithaeufigkeit": h_ltg110 * ltg_l, "ausfalldauer": ta_ltg110},
    {"name": "HTR11", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HTR12", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HTR21", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HTR22", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QTR1", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QTR2", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HLS11", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "HLS12", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "T1", "defizithaeufigkeit": h_T, "ausfalldauer": ta_T},
    {"name": "T2", "defizithaeufigkeit": h_T, "ausfalldauer": ta_T},
    {"name": "HLS21", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "HLS22", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "HTR31", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HTR32", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
]

# Erstellen des ersten DataFrames
df_h_schaltung = pd.DataFrame(elemente_H_Schaltung)
df_h_schaltung = df_h_schaltung.rename(
    columns={
        "name": "Ausgefallenes Element",
        "defizithaeufigkeit": "H [1/a]",
        "ausfalldauer": "Ausfalldauer [h]",
    }
)

# Erstellen des zweiten DataFrames
rows = [
    ["LTG1", "LT", "V1 + V2", "-"],
    ["LTG2", "LT", "V1 + V2", "-"],
    ["HTR11", "TR", "V1 + V2", "V1"],
    ["HTR12", "TR", "V1 + V2", "V2"],
    ["HTR21", "TR", "V1 + V2", "V1"],
    ["HTR22", "TR", "V1 + V2", "V2"],
    ["QTR1", "TR", "V1 + V2", "V1"],
    ["QTR2", "TR", "V1 + V2", "V2"],
    ["HLS11", "LS", "V1 + V2", "V1"],
    ["HLS12", "LS", "V1 + V2", "V2"],
    ["T1", "T", "V1", "V1"],
    ["T2", "T", "V2", "V2"],
    ["HLS21", "LS", "V1", "V1"],
    ["HLS22", "LS", "V2", "V2"],
    ["HTR31", "TR", "V1", "V1"],
    ["HTR32", "TR", "V2", "V2"],
]

df = pd.DataFrame(
    rows,
    columns=[
        "Ausgefallenes Element",
        "Zuverlässigkeitskenn­größe, Typ",
        "Ausgefallene Verbraucher - Nach Fehler",
        "Ausgefallene Verbraucher - Nach Umschaltung",
    ],
)

# Zusammenfügen der DataFrames anhand der Spalte "Ausgefallenes Element"
df_gesamt = pd.merge(df_h_schaltung, df, on="Ausgefallenes Element", how="left")

# Ausgabe des kombinierten DataFrames
print(tabulate(df_gesamt, headers="keys", tablefmt="grid"))

+----+-------------------------+-----------+--------------------+----------------------------------+------------------------------------------+-----------------------------------------------+
|    | Ausgefallenes Element   |   H [1/a] |   Ausfalldauer [h] | Zuverlässigkeitskenn­größe, Typ   | Ausgefallene Verbraucher - Nach Fehler   | Ausgefallene Verbraucher - Nach Umschaltung   |
+====+=========================+===========+====================+==================================+==========================================+===============================================+
|  0 | LTG1                    |  0.125    |                 80 | LT                               | V1 + V2                                  | -                                             |
+----+-------------------------+-----------+--------------------+----------------------------------+------------------------------------------+-----------------------------------------------+
|  1 | LTG2                    |  0.125

In [ ]:
# Aktuelle Situation: Kunde hängt auf V1 an UW A (H-Schaltung), Zusätzliche Freileitung zu Großkunde nicht existent
# Alle Zeilen der Spalte "Ausgefallene Verbraucher - Nach Fehler" herausfiltern, die V1 enthalten

# Filtern der Zeilen, die "V1" in der Spalte "Ausgefallene Verbraucher - Nach Fehler" enthalten
df_result = df_gesamt[
    df_gesamt["Ausgefallene Verbraucher - Nach Fehler"].str.contains("V1", na=False)
].copy()

# ----------- NÄCHSTE SCHRITTE
# Zuverlässigkeitskennwerte für jedes Element berechnen und alle Werte addieren


# Berechnung der Nichtverlässlichkeit (Ausfallhäufigkeit * Ausfalldauer)
df_result["Nichtverlässlichkeit"] = df_result["H [1/a]"] * df_result["Ausfalldauer [h]"]
nv_sum = df_result["Nichtverlässlichkeit"].sum()


# Berechnung der Defizitleistung
df_result["Defizitleistung [MW/a]"] = df_result["H [1/a]"] * P_A
P_def_sum = df_result["Defizitleistung [MW/a]"].sum()

# Berechnung der Defizitenergie
df_result["Defizitenergie [MWh/a]"] = (
    df_result["H [1/a]"] * P_A * df_result["Ausfalldauer [h]"]
)
E_def_sum = df_result["Defizitenergie [MWh/a]"].sum()

# Ausgabe des aktualisierten DataFrames
print(tabulate(df_result, headers="keys", tablefmt="grid"))

# Addieren der jeweiligen Spalten und ausgeben

# Alle Zeilen der Spalte "Ausgefallene Verbraucher - Nach Umschaltung" herausfiltern, die V1 enthalten
df_result_V1nU = df_gesamt[
    df_gesamt["Ausgefallene Verbraucher - Nach Umschaltung"].str.contains(
        "V1", na=False
    )
].copy()

print(tabulate(df_result_V1nU, headers="keys", tablefmt="grid"))

# so wie im ersten Fall vorgehen

# Selbe Vorgehensweise wie bei der H-Schaltung

+----+-------------------------+-----------+--------------------+----------------------------------+------------------------------------------+-----------------------------------------------+------------------------+--------------------------+
|    | Ausgefallenes Element   |   H [1/a] |   Ausfalldauer [h] | Zuverlässigkeitskenn­größe, Typ   | Ausgefallene Verbraucher - Nach Fehler   | Ausgefallene Verbraucher - Nach Umschaltung   |   Nichtverlässlichkeit |   Defizitleistung [MW/a] |
+====+=========================+===========+====================+==================================+==========================================+===============================================+========================+==========================+
|  0 | LTG1                    |  0.125    |                 80 | LT                               | V1 + V2                                  | -                                             |              10        |                 0.625    |
+----+-----------------

In [49]:
# Berechnung für dritten Fall: Umbau von UW A auf Doppelsammelschienen-Topologie


elemente_H_Schaltung = [
    {
        "name": "LTG1",
        "defizithaeufigkeit": h_ltg110,
        "ausfalldauer": ta_ltg110,
        "laenge": ltg_l,
    },
    {
        "name": "LTG2",
        "defizithaeufigkeit": h_ltg110,
        "ausfalldauer": ta_ltg110,
        "laenge": ltg_l,
    },
    {"name": "LTR11", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "LTR12", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "LS11", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "LS12", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "STR11", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR12", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR21", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR22", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QTR1", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QTR2", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QLS", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "SS1", "defizithaeufigkeit": h_SS, "ausfalldauer": ta_SS},
    {"name": "SS2", "defizithaeufigkeit": h_SS, "ausfalldauer": ta_SS},
    {"name": "STR31", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR32", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR41", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR42", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "LS21", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "LS22", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "T1", "defizithaeufigkeit": h_T, "ausfalldauer": ta_T},
    {"name": "T2", "defizithaeufigkeit": h_T, "ausfalldauer": ta_T},
    {"name": "LS31", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "LS32", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "ATR11", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "ATR12", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
]

# Erstellen eines pandas-dataframes
data = pd.DataFrame(columns=["Name", "H [1/a]", "NV [h/a]", "P [MW/a]", "E [MWh/a]"])

# Berechnung der Zuverlässigkeitskennzahlen und Hinzufügen zum DataFrame
for i, element in enumerate(elemente_H_Schaltung):
    H, NV, P, E = calculate_kenngroessen(
        element["defizithaeufigkeit"],
        element["ausfalldauer"],
        P_A,
        element.get("laenge", 1),
    )

    # Hinzufügen der Daten zum DataFrame
    data.loc[i] = [element["name"], H, NV, P, E]

print(data)

# Beschreibung der Ausfallwirkung für die Doppelsammelschienen-Schaltanlage in einer Tabelle
rows = []

# Der in der Abbildung ersichtliche Schaltzustand wird als Basis für nachfolgende Überlegungen getroffen

# Zeilenweise Eintragen der Werte
rows.append(["LTG1", "LT", "-", "-"])
rows.append(["LTG2", "LT", "-", "-"])
rows.append(["LTR11", "TR", "-", "-"])
rows.append(["LTR12", "TR", "-", "-"])
rows.append(
    ["LS11", "LS", "V1", "-"]
)  # Annahme Fehlerbereich freigeschaltet (Kupplung, LTG), dann Trenner geöffnet + Kupplung zu → Wiederversorgung V1
rows.append(["LS12", "LS", "V2", "-"])


rows.append(["STR11", "TR", "V1", "-"])
rows.append(
    ["STR12", "TR", "V1 + V2", "-"]
)  # Wenn am Anfang beide unversorgt sind, kann ein benötigter Sammelschienenwechsel spannungsfrei durchgeführt werden

rows.append(["STR21", "TR", "V1 + V2", "-"])  # Gegengleich zu STR12
rows.append(["STR22", "TR", "V2", "-"])  # Gegengleich zu STR11

rows.append(["QTR1", "TR", "V1", "-"])
rows.append(["QTR2", "TR", "V2", "-"])
rows.append(["QLS", "LS", "V1 + V2", "-"])

rows.append(["SS1", "SS", "V1", "-"])
rows.append(["SS2", "SS", "V2", "-"])

rows.append(["STR31", "TR", "V1", "V1"])
rows.append(["STR32", "TR", "V1 + V2", "-"])

rows.append(["STR41", "TR", "V1 + V2", "-"])
rows.append(["STR42", "TR", "V2", "V2"])

rows.append(["LS21", "LS", "V1", "V1"])
rows.append(["LS22", "LS", "V2", "V2"])

rows.append(["T1", "T", "V1", "V1"])
rows.append(["T2", "T", "V2", "V2"])

rows.append(["LS31", "LS", "V1", "V1"])
rows.append(["LS32", "LS", "V2", "V2"])

rows.append(["ATR11", "TR", "V1", "V1"])
rows.append(["ATR12", "TR", "V2", "V2"])

# DataFrame erstellen
df = pd.DataFrame(
    rows,
    columns=[
        "Ausgefallenes Element",
        "Zuverlässigkeitskenn­größe, Typ",
        "Ausgefallene Verbraucher - Nach Fehler",
        "Ausgefallene Verbraucher - Nach Umschaltung",
    ],
)

# Tabelle ausgeben
# print(tabulate(df, headers="keys", tablefmt="grid"))

# Umbenennen der 'Name'-Spalte für das Merge
data_renamed = data.rename(columns={"Name": "Ausgefallenes Element"})

# Merge durchführen
df = df.merge(
    data_renamed[["Ausgefallenes Element", "H [1/a]"]],
    on="Ausgefallenes Element",
    how="left",
)

# Spalten neu anordnen: 'H [1/a]' an zweite Position verschieben
cols = df.columns.tolist()
cols.insert(1, cols.pop(cols.index("H [1/a]")))
df = df[cols]

# Tabelle ausgeben
print(tabulate(df, headers="keys", tablefmt="grid"))

# Erstens: V1 aus Spalte "Ausgefallenes Element - Nach Fehler" herausfiltern

     Name   H [1/a]   NV [h/a]  P [MW/a]  E [MWh/a]
0    LTG1  0.125000  10.000000  0.625000   50.00000
1    LTG2  0.125000  10.000000  0.625000   50.00000
2   LTR11  0.003620   0.260640  0.018100    1.30320
3   LTR12  0.003620   0.260640  0.018100    1.30320
4    LS11  0.000259   0.018648  0.001295    0.09324
5    LS12  0.000259   0.018648  0.001295    0.09324
6   STR11  0.003620   0.260640  0.018100    1.30320
7   STR12  0.003620   0.260640  0.018100    1.30320
8   STR21  0.003620   0.260640  0.018100    1.30320
9   STR22  0.003620   0.260640  0.018100    1.30320
10   QTR1  0.003620   0.260640  0.018100    1.30320
11   QTR2  0.003620   0.260640  0.018100    1.30320
12    QLS  0.000259   0.018648  0.001295    0.09324
13    SS1  0.000148   0.010656  0.000740    0.05328
14    SS2  0.000148   0.010656  0.000740    0.05328
15  STR31  0.003620   0.260640  0.018100    1.30320
16  STR32  0.003620   0.260640  0.018100    1.30320
17  STR41  0.003620   0.260640  0.018100    1.30320
18  STR42  0